In [3]:
import torch
import sys
sys.path.insert(0, './models')
# Option 1) Load the model directly.
# (this may print warning based on the installed version of python)model = torch.load('ggcnn_weights_cornell/ggcnn_epoch_23_cornell')


In [6]:
model = torch.load('/home/riot/kinova_gen3_lite/src/ggcnn/scripts/models/pretrained/ggcnn_epoch_23_cornell')

/home/riot/kinova_gen3_lite/src/ggcnn_env/lib/python3.8/site-packages/torch/serialization.py:1113: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/riot/kinova_gen3_lite/src/ggcnn_env/lib/python3.8/site-packages/torch/serialization.py:1113: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [12]:
import rospy
from kortex_driver.msg import BaseCyclic_Feedback
rospy.init_node('test')

In [13]:
robot_state = rospy.wait_for_message("/kinova_gen3_lite/base_feedback", BaseCyclic_Feedback, timeout=2)

In [15]:
type(robot_state)

kortex_driver.msg._BaseCyclic_Feedback.BaseCyclic_Feedback

In [ ]:
for s in FrankaErrors.__slots__:
    if getattr(msg.current_errors, s):
        self.stop()
        if not self.ROBOT_ERROR_DETECTED:
            rospy.logerr('Robot Error Detected')
        self.ROBOT_ERROR_DETECTED = True

In [47]:
def robot_state_callback(msg: BaseCyclic_Feedback):
    robot_state = msg
    for s in BaseCyclic_Feedback.__slots__:
        if s == "frame_id":
            continue
        if s == "base" or s == "interconnect":
            if getattr(robot_state,s).fault_bank_a or getattr(robot_state,s).fault_bank_b:
                print("robot error")
        else:
            if any (actuator.fault_bank_a or actuator.fault_bank_b for actuator in getattr(robot_state,s)):
                print("robot error")


In [48]:
rospy.Subscriber('/kinova_gen3_lite/base_feedback', BaseCyclic_Feedback ,robot_state_callback, queue_size=1)

    